In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bladder_Cancer"
cohort = "GSE245953"

# Input paths
in_trait_dir = "../../input/GEO/Bladder_Cancer"
in_cohort_dir = "../../input/GEO/Bladder_Cancer/GSE245953"

# Output paths
out_data_file = "../../output/preprocess/Bladder_Cancer/GSE245953.csv"
out_gene_data_file = "../../output/preprocess/Bladder_Cancer/gene_data/GSE245953.csv"
out_clinical_data_file = "../../output/preprocess/Bladder_Cancer/clinical_data/GSE245953.csv"
json_path = "../../output/preprocess/Bladder_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression data from muscle-invasive bladder cancer samples II"
!Series_summary	"Gene signatures based on the median expression of a preselected set of genes can provide prognostic and treatment outcome prediction and so be valuable clinically."
!Series_summary	"Different health care services use different gene expression platforms to derive gene expression data. Here we have derived gene expression data using a microarray platform."
!Series_overall_design	"RNA extracted from FFPE blocks from patients with muscle-invasive bladder cancer and full transcriptome analysis on Clariom S microarray platform."
Sample Characteristics Dictionary:
{0: ['condition: Muscle-invasive bladder cancer']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# From the background info, it appears there is gene expression data
# The series mentions "Gene expression data using a microarray platform"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Looking at the sample characteristics dict, we only have one key (0)
# Trait (Bladder Cancer): row 0 contains cancer status - all samples are MIBC
trait_row = 0

# Age: Not available in the sample characteristics
age_row = None

# Gender: Not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert trait values to binary.
    For Bladder Cancer: 1 for having cancer, 0 for control.
    """
    if value is None:
        return None
    
    # Extract the part after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # All samples are cancer cases (muscle-invasive bladder cancer)
    if "muscle-invasive bladder cancer" in value.lower():
        return 1
    return None  # Unknown/unclear values

def convert_age(value):
    """
    Convert age values to continuous numeric format.
    """
    if value is None:
        return None
    
    # Extract the part after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Try to convert to float (handles both integers and decimals)
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """
    Convert gender values to binary format where:
    0 = Female
    1 = Male
    """
    if value is None:
        return None
    
    # Extract the part after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.lower()
    
    if value in ['male', 'm']:
        return 1
    elif value in ['female', 'f']:
        return 0
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available (if trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait_row is not None, extract clinical features
if trait_row is not None:
    # Load clinical data from previous step
    clinical_data = pd.DataFrame({0: ['condition: Muscle-invasive bladder cancer']})
    
    # Extract clinical features using the provided function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical dataframe
    preview = preview_df(selected_clinical_df)
    print("Selected Clinical Features Preview:", preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical features to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Selected Clinical Features Preview: {0: [1.0]}
Clinical data saved to ../../output/preprocess/Bladder_Cancer/clinical_data/GSE245953.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['AFFX-BkGr-GC03_st', 'AFFX-BkGr-GC04_st', 'AFFX-BkGr-GC05_st',
       'AFFX-BkGr-GC06_st', 'AFFX-BkGr-GC07_st', 'AFFX-BkGr-GC08_st',
       'AFFX-BkGr-GC09_st', 'AFFX-BkGr-GC10_st', 'AFFX-BkGr-GC11_st',
       'AFFX-BkGr-GC12_st', 'AFFX-BkGr-GC13_st', 'AFFX-BkGr-GC14_st',
       'AFFX-BkGr-GC15_st', 'AFFX-BkGr-GC16_st', 'AFFX-BkGr-GC17_st',
       'AFFX-BkGr-GC18_st', 'AFFX-BkGr-GC19_st', 'AFFX-BkGr-GC20_st',
       'AFFX-BkGr-GC21_st', 'AFFX-BkGr-GC22_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are Affymetrix microarray probe IDs (AFFX prefix), not human gene symbols.
# They need to be mapped to standard gene symbols for analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 // ENSEMBL // olfactory receptor, family 4, subfamily F, member 5 [gene_biotype:protein_coding transcript_biotype:protein_coding] // chr1 // 10

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Looking at the gene annotation columns and gene expression data indexes
# The gene expression data has index values like 'AFFX-BkGr-GC03_st', etc.
# In the gene annotation dataframe:
# - 'ID' contains values like 'TC0100006437.hg.1' which don't match our gene expression indexes
# - 'SPOT_ID.1' seems to contain gene information in a complex format

# We need to extract the relationship between probe IDs and gene symbols
# First, let's check if our gene expression IDs match any of the columns in gene_annotation
# Since we couldn't find a direct match in the preview, we need to check if the gene_data's index values
# are present in the gene_annotation dataframe

# Check a few probe IDs from gene_data
print("First few probe IDs from gene expression data:")
print(gene_data.index[:5])

# Let's check if any column in gene_annotation might contain information about gene symbols
# We need to extract gene symbols from the 'SPOT_ID.1' column which contains RefSeq annotations
# Create a mapping function that will extract gene symbols from the complex annotation text

# Create a mapping dataframe with probe IDs and gene symbols
# We'll use the ID column as probe ID and extract gene symbols from SPOT_ID.1
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'SPOT_ID.1')
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("Gene expression data after mapping:")
print(gene_data.shape)
print(gene_data.index[:20])  # Print first 20 gene symbols


First few probe IDs from gene expression data:
Index(['AFFX-BkGr-GC03_st', 'AFFX-BkGr-GC04_st', 'AFFX-BkGr-GC05_st',
       'AFFX-BkGr-GC06_st', 'AFFX-BkGr-GC07_st'],
      dtype='object', name='ID')


Gene mapping preview:
                  ID                                               Gene
0  TC0100006437.hg.1  NM_001005484 // RefSeq // Homo sapiens olfacto...
1  TC0100006476.hg.1  NM_152486 // RefSeq // Homo sapiens sterile al...
2  TC0100006479.hg.1  NM_198317 // RefSeq // Homo sapiens kelch-like...
3  TC0100006480.hg.1  NM_001160184 // RefSeq // Homo sapiens pleckst...
4  TC0100006483.hg.1  NM_005101 // RefSeq // Homo sapiens ISG15 ubiq...


Gene expression data after mapping:
(84779, 310)
Index(['A-', 'A-1', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0',
       'A0A075B6T3', 'A0A075B739', 'A0A075B767', 'A0A087WSY0', 'A0A087WTG2',
       'A0A087WTH5', 'A0A087WV48', 'A0A087WWA1', 'A0A087WWL8', 'A0A087WWU0'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# First, normalize gene symbols using the function from the library
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene count: {len(gene_data)}")
print(f"Normalized gene count: {len(normalized_gene_data)}")

# Create directory for the gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data to a CSV file
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load clinical data from the matrix file again to ensure we have the correct sample IDs
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

print("Clinical data structure:")
print(clinical_data.shape)
print("First few rows of clinical data:")
print(clinical_data.head())

# Extract clinical features with the correct sample IDs
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Clinical data shape after extraction: {selected_clinical_df.shape}")
print("First few sample IDs in clinical data:")
print(list(selected_clinical_df.columns)[:5])
print("First few sample IDs in gene data:")
print(list(normalized_gene_data.columns)[:5])

# Check for column overlap
common_samples = set(selected_clinical_df.columns).intersection(set(normalized_gene_data.columns))
print(f"Number of common samples between clinical and gene data: {len(common_samples)}")

# Save the clinical data for inspection
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Check if linking was successful
if len(linked_data) == 0 or trait not in linked_data.columns:
    print("Linking clinical and genetic data failed - no valid rows or trait column missing")
    
    # Check what columns are in the linked data
    if len(linked_data.columns) > 0:
        print("Columns in linked data:")
        print(list(linked_data.columns)[:10])  # Print first 10 columns
    
    # Set is_usable to False and save cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,  # Consider it biased if linking fails
        df=pd.DataFrame({trait: [], 'Gender': []}), 
        note="Data linking failed - unable to match sample IDs between clinical and gene expression data."
    )
    print("The dataset was determined to be not usable for analysis.")
else:
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 5. Conduct quality check and save the cohort information.
    note = "Dataset contains gene expression data from bladder cancer samples with molecular subtyping information."
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=linked_data, 
        note=note
    )
    
    # 6. If the linked data is usable, save it as a CSV file.
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("The dataset was determined to be not usable for analysis due to bias in the trait distribution.")

Original gene count: 84779
Normalized gene count: 19858


Normalized gene data saved to ../../output/preprocess/Bladder_Cancer/gene_data/GSE245953.csv


Clinical data structure:
(1, 311)
First few rows of clinical data:
         !Sample_geo_accession                                 GSM7851882  \
0  !Sample_characteristics_ch1  condition: Muscle-invasive bladder cancer   

                                  GSM7851883  \
0  condition: Muscle-invasive bladder cancer   

                                  GSM7851884  \
0  condition: Muscle-invasive bladder cancer   

                                  GSM7851885  \
0  condition: Muscle-invasive bladder cancer   

                                  GSM7851886  \
0  condition: Muscle-invasive bladder cancer   

                                  GSM7851887  \
0  condition: Muscle-invasive bladder cancer   

                                  GSM7851888  \
0  condition: Muscle-invasive bladder cancer   

                                  GSM7851889  \
0  condition: Muscle-invasive bladder cancer   

                                  GSM7851890  ...  \
0  condition: Muscle-invasive bladder cancer  

Linked data shape after handling missing values: (310, 19859)
Quartiles for 'Bladder_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Bladder_Cancer' in this dataset is severely biased.

The dataset was determined to be not usable for analysis due to bias in the trait distribution.
